In [1]:
import numpy as np
import pandas as pd
import pandapower as pp

# Configurar opciones de impresión de NumPy y pandas
np.set_printoptions(precision=4, suppress=True, linewidth=100)
pd.options.display.float_format = '{:.4f}'.format

# Creación de una red eléctrica vacía
net = pp.create_empty_network()

# Crear barras
b1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
b2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
b1a = pp.create_bus(net, vn_kv=220, name="Barra 1A")
b2a = pp.create_bus(net, vn_kv=220, name="Barra 2A")
b3a = pp.create_bus(net, vn_kv=220, name="Barra 3A")
b1b = pp.create_bus(net, vn_kv=220, name="Barra 1B")
b2b = pp.create_bus(net, vn_kv=220, name="Barra 2B")

# Conexión de malla y transformador
pp.create_ext_grid(net, bus=b1, vm_pu=1, name="Conexion de malla")
pp.create_transformer(net, hv_bus=b2, lv_bus=b1, std_type="100 MVA 220/110 kV")

# Creación de líneas entre barras
pp.create_line(net, from_bus=b2, to_bus=b1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2-B1A")
pp.create_line(net, from_bus=b2, to_bus=b1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2-B1B")
pp.create_line(net, from_bus=b1a, to_bus=b2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B1A-B2A")
pp.create_line(net, from_bus=b2a, to_bus=b3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2A-B3A")
pp.create_line(net, from_bus=b3a, to_bus=b2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B3A-B2B")
pp.create_line(net, from_bus=b2b, to_bus=b1b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Línea B2B-B1B")

# Valores nominales de carga y creación de cargas
P_nominal_MW = 150
Q_nominal_MVA = 100
bus_dict = {"b1a": pp.get_element_index(net, "bus", "Barra 1A"), "b2a": pp.get_element_index(net, "bus", "Barra 2A"),
            "b3a": pp.get_element_index(net, "bus", "Barra 3A"), "b2b": pp.get_element_index(net, "bus", "Barra 2B"),
            "b1b": pp.get_element_index(net, "bus", "Barra 1B")}
cargas = {"b1a": 0.20, "b2a": 0.35, "b3a": 0.15, "b2b": 0.60, "b1b": 0.10}
for barra, porcentaje in cargas.items():
    P = P_nominal_MW * porcentaje
    Q = Q_nominal_MVA * porcentaje
    pp.create_load(net, bus=bus_dict[barra], p_mw=P, q_mvar=Q, name=f"Carga en {barra}")

# Flujo de potencia
pp.runpp(net, max_iteration=100, tolerance_mva=1e-9)
# Imprimir resultados de simulación
print("\nResultados de las Barras:")
print(net.res_bus)

# Obtener la matriz de admitancia Ybus
Ybus = net._ppc['internal']['Ybus'].todense()
Y_real = np.real(Ybus)
Y_imag = np.imag(Ybus)

# Potencias inyectadas en las barras PQ (MW y MVAR)
P_inyectado = np.array([0.2*P, 0.35*P, 0.15*P, 0.6*P, 0.1*P])  # MW
Q_inyectado = np.array([0.2*Q, 0.35*Q, 0.15*Q, 0.6*Q, 0.1*Q])  # MVAR

# Valores iniciales
V_ini = np.ones(5)
theta_ini = np.zeros(5)


# Función para calcular las derivadas
def calcular_derivadas(V, theta, Y_real, Y_imag):
    n = len(V)
    H = np.zeros((n, n))
    N = np.zeros((n, n))
    M = np.zeros((n, n))
    L = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                H[i, j] = -V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
                N[i, j] = -V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                M[i, j] = V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                L[i, j] = -V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
            else:
                H[i, i] = V[i]**2 *(V[i]*np.sum(V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))))
                N[i, i] = -V[i] * Y_real[i, i] - (np.sum(V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))) / V[i])
                M[i, i] = V[i]**2 * Y_real[i, i] - np.sum(V[i] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j])))
                L[i, i] = V[i] * Y_imag[i, i] - (np.sum(V[i] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))) / V[i])

    return H, N, M, L

# Función para calcular las funciones de error
def calcular_f(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado):
    n = len(V)
    delta_P = np.zeros(n)
    delta_Q = np.zeros(n)

    for i in range(n):
        sum_P = 0.0
        sum_Q = 0.0
        for j in range(n):
            sum_P +=  V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
            sum_Q +=  V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))

        delta_P[i] = P_inyectado[i] - (V[i] *sum_P)
        delta_Q[i] = Q_inyectado[i] - (V[i] *sum_Q)

    f = np.concatenate((delta_P, delta_Q))
    return f

# Función para el método de Newton-Raphson
def newton_raphson(V, theta, Y_real, Y_imag, P_inyectado, Q_inyectado, tol=1e-9, max_iter=100):
    iter = 0
    error = tol + 1

    while error > tol and iter < max_iter:
        H, N, M, L = calcular_derivadas(V, theta, Y_real, Y_imag)
        # Calcular las potencias activa y reactiva inyectadas en cada barra
        P_calc = np.zeros(len(V))
        Q_calc = np.zeros(len(V))
        for i in range(len(V)):
            for j in range(len(V)):
                P_calc[i] += V[i] * V[j] * (Y_real[i, j] * np.cos(theta[i] - theta[j]) + Y_imag[i, j] * np.sin(theta[i] - theta[j]))
                Q_calc[i] += V[i] * V[j] * (Y_real[i, j] * np.sin(theta[i] - theta[j]) - Y_imag[i, j] * np.cos(theta[i] - theta[j]))
        # Calcular el vector de desequilibrio f
        f_P = P_inyectado - P_calc
        f_Q = Q_inyectado - Q_calc
        f = np.concatenate((f_P, f_Q))

        J1 = np.block([
            [H, N],
            [M, L]
        ])

        incremento = np.linalg.solve(J1, -f)
        delta_theta = incremento[:len(theta)]
        delta_V = incremento[len(theta):]

        theta += delta_theta
        V += delta_V

        error = np.max(np.abs(f))
        iter += 1

    if error > tol:
        print(f'El método de Newton-Raphson no convergió después de {max_iter} iteraciones.')

    return V, theta  # Convertir theta de radianes a grados

# Aplicar el método de Newton-Raphson
V_resultado, theta_resultado = newton_raphson(V_ini, np.radians(theta_ini), Y_real, Y_imag, P_inyectado, Q_inyectado)

# Mostrar resultados
print("Resultados del método de Newton-Raphson:")
print("Voltajes (en p.u.):", V_resultado)
print("Ángulos (en grados):", theta_resultado)



Resultados de las Barras:
   vm_pu  va_degree      p_mw  q_mvar
0 1.0000     0.0000 -212.3102  5.8073
1 1.0334   -14.2765    0.0000  0.0000
2 1.0315   -14.4923   30.0000 20.0000
3 1.0292   -14.7303   52.5000 35.0000
4 1.0284   -14.8649   22.5000 15.0000
5 1.0324   -14.4698   15.0000 10.0000
6 1.0274   -14.8735   90.0000 60.0000
El método de Newton-Raphson no convergió después de 100 iteraciones.
Resultados del método de Newton-Raphson:
Voltajes (en p.u.): [0.3604 0.0144 0.0049 0.0193 0.0095]
Ángulos (en grados): [-20.8908  -0.374   -3.7451  -0.8089  -3.7081]
